## Training and Prediction Codes for Relation Extraction Models 

In [2]:
!pip3 install transformers==3.5.1

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import sys
sys.path.insert(0, '../')
import pandas as pd
from importlib import reload
import numpy as np
from pprint import pprint
import re
from collections import defaultdict
import json
import pickle

In [4]:
import src.data.config as cf
import src.data.read_data as rd
import src.data.preprocess as pr
import src.models.train_binary_classification as clf
import src.models.prediction as pred
reload(clf)

cuda
cuda
cuda
cuda


<module 'src.models.train_binary_classification' from '../src/models/train_binary_classification.py'>

## Read data set

Read training set with the following code:

In [25]:
df_ent, df_abst, df_relation, df_gold = rd.read_all_data('train')
df_abst.head()

,article_id,title,abstract
0,16357751,Selective costimulation modulators: a novel ap...,T cells have a central role in the orchestrati...
1,14967461,Emerging role of epidermal growth factor recep...,Combination chemotherapy regimens have emerged...
2,23468099,Effects of chronic social defeat stress on beh...,RATIONALE: Social defeat stress induces physio...
3,23293962,Hepatocyte growth factor activator inhibitor t...,"Matriptase-2, a recently identified cell surfa..."
4,7678677,Alprenolol and bromoacetylalprenololmenthane a...,We studied the effects of alprenolol and bromo...


Read the sentences that were splitted with GENIA sentence splitter

In [26]:
with open('../pickles/geniass_sentence_split_frame.pickle', 'rb') as handle:
       df_sentences = pickle.load(handle)

### Train a relation extraction model with train data 

In [27]:
df_final = pr.preprocess('train')
print(len(df_final))
df_final.head()

/home/akoksal/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/akoksal/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/akoksal/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

18046


,article_id,sent_id,sentence,start_index,end_index,arg1,entity_mention_1,start_offset_1,end_offset_1,type_1,...,ARG1,ARG2,ARG1_num,ARG2_num,label,chem_1,chem_2,pro_1,pro_2,changed_sent
0,16357751,6,Recent studies have provided consistent eviden...,1137,1395,T1,methotrexate,1342.0,1354.0,CHEMICAL,...,Arg1:T1,Arg2:T2,1,2,11,204,216,226,247,Recent studies have provided consistent eviden...
1,14967461,8,Epidermal growth factor receptor inhibitors cu...,1174,1407,T1,gefitinib,1277.0,1286.0,CHEMICAL,...,Arg1:T1,Arg2:T22,1,22,4,102,111,0,32,<e2> Epidermal growth factor receptor </e2> ...
10,14967461,8,Epidermal growth factor receptor inhibitors cu...,1174,1407,T2,Iressa,1288.0,1294.0,CHEMICAL,...,Arg1:T2,Arg2:T22,2,22,4,113,119,0,32,<e2> Epidermal growth factor receptor </e2> ...
19,14967461,8,Epidermal growth factor receptor inhibitors cu...,1174,1407,T3,ZD1839,1296.0,1302.0,CHEMICAL,...,Arg1:T3,Arg2:T22,3,22,4,121,127,0,32,<e2> Epidermal growth factor receptor </e2> ...
28,14967461,8,Epidermal growth factor receptor inhibitors cu...,1174,1407,T4,erlotinib,1308.0,1317.0,CHEMICAL,...,Arg1:T4,Arg2:T22,4,22,4,133,142,0,32,<e2> Epidermal growth factor receptor </e2> ...


Train BioBERT-based relation extraction model on biomedical data set

In [68]:
clf.train_model('','12092020', save_dir = "../models/model_28082020_2_classes.pt", learning_rate=0.00002, wd=0.3)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Loss:  0.5769571747098651
Acc:  0.6173469387755102
Micro F1:  0.61875
Loss:  0.5323309838169753
Acc:  0.6369949494949495
Micro F1:  0.6375
Loss:  0.5015170634192908
Acc:  0.6673657718120806
Micro F1:  0.6679166666666667
Loss:  0.4779245603324181
Acc:  0.6846733668341709
Micro F1:  0.685
Loss:  0.44867800385597717
Acc:  0.7078313253012049
Micro F1:  0.70725
Loss:  0.4264776344133859
Acc:  0.7259615384615384
Micro F1:  0.7264583333333332
Loss:  0.40977345908695784
Acc:  0.7397922636103151
Micro F1:  0.739642857142857
Loss:  0.39980025040475947
Acc:  0.7478070175438597
Micro F1:  0.7478125
Loss:  0.3906312045415154
Acc:  0.755011135857461
Micro F1:  0.755138888888889
Loss:  0.3845180813857692
Acc:  0.7588927855711423
Micro F1:  0.759375
Loss:  0.3730179113361353
Acc:  0.76775956284153
Micro F1:  0.7678409090909091
Loss:  0.36579431548640007
Acc:  0.7750417362270451
Micro F1:  0.7754166666666665
Loss:  0.3591034330129164
Acc:  0.7812018489984591
Micro F1:  0.7814423076923077
Loss:  0.35387

/home/akoksal/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/akoksal/.local/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Loss:  nan
Acc:  nan
Micro F1:  0.8712500000000001


/home/akoksal/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/akoksal/.local/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

Loss:  nan
Acc:  nan
Micro F1:  0.8816666666666667


/home/akoksal/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/akoksal/.local/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Loss:  nan
Acc:  nan
Micro F1:  0.88125


Epoch:  50%|█████     | 2/4 [36:20<36:20, 1090.01s/it]


KeyboardInterrupt: 

## Predict relations between chemical and protein entities with the trained model

In [10]:
write_file = 'test_class_2_30082021'
df_pred = pred.prediction(write_file, save_dir="../models/model_28082020_2_classes.pt", batch_size=16)
df_pred.head()

,article_id,sent_id,sentence,start_index,end_index,arg1,entity_mention_1,start_offset_1,end_offset_1,type_1,...,ARG2,ARG1_num,ARG2_num,label,diff_chem_1,diff_chem_2,diff_pro_1,diff_pro_2,changed_sent,Pred
0,10076535,0,Androgen antagonistic effect of estramustine p...,0,115,T44,Androgen,0.0,8.0,CHEMICAL,...,Arg2:T63,44,63,6,0,8,98,115,<e1> Androgen </e1> antagonistic effect of e...,0
1,10076535,0,Androgen antagonistic effect of estramustine p...,0,115,T45,estramustine phosphate,32.0,54.0,CHEMICAL,...,Arg2:T63,45,63,6,32,54,98,115,Androgen antagonistic effect of <e1> estramus...,1
2,10076535,0,Androgen antagonistic effect of estramustine p...,0,115,T46,EMP,56.0,59.0,CHEMICAL,...,Arg2:T63,46,63,6,56,59,98,115,Androgen antagonistic effect of estramustine p...,1
3,10076535,0,Androgen antagonistic effect of estramustine p...,0,115,T47,androgen,98.0,106.0,CHEMICAL,...,Arg2:T63,47,63,11,98,106,98,115,Androgen antagonistic effect of estramustine p...,0
4,10076535,6,"We investigated, by displacement of [3H]methyl...",600,1098,T32,[3H]methyltrienolone,637.0,657.0,CHEMICAL,...,Arg2:T59,32,59,11,36,56,342,359,"We investigated, by displacement of <e1> [3H]...",0
